# **Interpreting ML Models with LIME: A Hands-on Workshop**
In this workshop, we'll explore how to use Local Interpretable Model-Agnostic Explanations (LIME) to interpret machine learning models, both for tabular and image data. This workshop was created for ELE392, taught by Dr. Chiovaro at the University of Rhode Island

**Topics Covered:**

*  Using LIME for Tabular Classification
*  Using LIME for Image Explanation
*  Using LIME for Text Explanation

**Switch to GPU first if available.**

In [ ]:
# !pip install lime pandas tqdm scikit-learn matplotlib Pillow torch torchvision transformers datasets
!pip install lime tqdm Pillow transformers datasets

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm #Ben found this

import torch
import torch.nn as nn
import torch.optim as optim

from lime import lime_tabular, lime_image
from PIL import Image
import torchvision.models as models
import torchvision.transforms as T
from skimage.segmentation import mark_boundaries

## **Part 1: LIME for Tabular Data**

###**Training Model on IRIS Dataset:**

We will use logistic regression to classify the Iris dataset and use LIME to interpret the predictions.

The model implementation and training is the same as any other lab we have previously done.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df['target_name'] = df['target'].apply(lambda i: iris.target_names[i])
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = df[iris.feature_names].values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)

model = nn.Linear(X_train.shape[1], len(iris.target_names))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in tqdm(range(200), desc='Training'):
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
X_test_t = torch.tensor(X_test, dtype=torch.float32)
with torch.no_grad():
    logits = model(X_test_t)
    preds = logits.argmax(dim=1).numpy()
    acc = (preds == y_test).mean()
print(f'Test Accuracy: {acc * 100:.2f}%')

###**Using LIME to Explain Model:**

After you have a trained model, LIME can be implemented to explain its predictions. This will be done in 3 main parts.



####**1) Create Explainer:**

This section initializes the explainer using LimeTabularExplainer()

There are 4 parameters needed.



1.   **training_data**

  The 2d numpy array of the training data used by the model.

2.   **mode**

  Either 'classification' or 'regression'

3.   **feature_names**

  List of names (strings) of the features

4.   **class_names**

  List of names of the classes (targets)




####**2) Define Prediction Function:**

  For *classifiers,* this should be a function that takes a numpy array and outputs prediction probabilities.

  For *regressors,* this takes a numpy array and returns the predictions.



####**3) Create Explanation:**

Call explain_instance() from the explainer.

This uses the following parameters:

1.  **data_row**

  1d numpy array corresponding to a row of input data.

2.  **predict_fn**

  Prediction function.

3.  **num_features**

  The maximum number of features present in the explanation.

There are multiple ways to display the explanation including:

*  as_html()
*  as_list()
*  as_map()
*  as_pyplot_figure()
* show_in_notebook()

In Colab, show_in_notebook() is the best way to view the explanations.

Additionally, the html explanation can be downloaded using save_to_file()

In [ ]:
# Create Explainer
explainer = lime_tabular.LimeTabularExplainer(
    X_train,
    feature_names=iris.feature_names,
    class_names=iris.target_names.tolist(),
    mode='classification')

In [ ]:
# Define Prediction Function
def predict_fn(input):
    input_t = torch.tensor(input, dtype=torch.float32)
    with torch.no_grad():
        logits = model(input_t)
        probs = torch.softmax(logits, dim=1)
    return probs.numpy()

In [ ]:
# Create Explanation
i = 0
exp = explainer.explain_instance(X_test[i], predict_fn, num_features=4)
exp.show_in_notebook(show_table=True)

## **Part 2: LIME for Image Classification**
We now use a pretrained ResNet50 model and LIME to highlight important regions in an image.

In [ ]:
# Get Image
!wget -O cat.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Felis_catus-cat_on_snow.jpg/640px-Felis_catus-cat_on_snow.jpg

In [ ]:
# Prep model and image

model_cnn = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model_cnn.eval()

transform = T.Compose([
    T.Resize(224),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

img = Image.open('cat.jpg')
plt.imshow(img); plt.axis('off')

###**Using LIME to Explain Image:**

LIME can be used to explain the most important features of an image when using a classification model. Similar to the previous example, there are 3 main parts:

####**1) Define Classifier Function:**

 This function takes a numpy array and outputs prediction probabilities.



####**2) Create Explainer:**

This section initializes the explainer using LimeImageExplainer()

The explain_instance() function is called. For this, here are 5 parameters needed.



1.   **image**

  The numpy array of the image.

2.   **classifier_fn**

  The classifier function defined above.

3.   **top_labels**

  Produce explanations for the n labels with highest prediction probabilities, where n is this parameter.

4.   **hide_color**

  The color in which the masked pixels are set to. (None or 0 to 255)

5.  **num_samples**

  Size of the neighborhood to learn the linear model.




####**3) Create Explanation:**

The get_image_and_mask() function will return the masked image created. 4 parameters are needed:

1.  **label**

  Label to explain

2.  **positive_only**

  If True, only take superpixels that have positively contributed to the prediction.
  
  (The negative_only parameter can be used to highlight the pixels that have negatively contributed)

3. **hide_rest**

  If true, make the non-explaining parts of the image gray.

4. **num_features**

  Number of superpixels to include in the explaination

In [ ]:
# Image Prediction Function
def predict_img(images):
    batch = torch.stack([transform(Image.fromarray(img)).to('cpu') for img in images])
    with torch.no_grad():
        logits = model_cnn(batch)
        probs = torch.nn.functional.softmax(logits, dim=1)
    return probs.cpu().numpy()

# Explain the image
image_explainer = lime_image.LimeImageExplainer()
explanation = image_explainer.explain_instance(
    np.array(img),
    predict_img,
    top_labels=1,
    hide_color=0,
    num_samples=1000)

# Get the masked image
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0],
    positive_only=True,
    hide_rest=True,
    num_features=5)

# Show the image
plt.imshow(mark_boundaries(temp / 255.0, mask)); plt.axis('off')

## **Part 3: LIME for Text Classification**

###**Importing Pre-Trained Model:**

We now use a pretrained RoBERTa model on the IMDB movie reviews dataset.

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from datasets import load_dataset
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import Counter
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, get_scheduler
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Set a random seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Load the IMDB dataset (already pre-split into train and test sets)
dataset = load_dataset("imdb")

# Reduce dataset size to save memory (use only 500 training samples)
train_texts = dataset['train']['text'][:500]
train_labels = dataset['train']['label'][:500]

# Use full test set for evaluation
test_texts = dataset['test']['text'][:100]
test_labels = dataset['test']['label'][:100]

# Print dataset size
print(f"Training samples: {len(train_texts)}")
print(f"Testing samples: {len(test_texts)}")

In [ ]:
model_name = "roberta-base"

# Load the tokenizer for DistilBERT (a smaller version of BERT)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to tokenize text
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=256, return_tensors="pt")

# Tokenize training and testing texts
train_encodings = tokenize_function(train_texts)
test_encodings = tokenize_function(test_texts)

# Convert labels into PyTorch tensors
train_labels_tensor = torch.tensor(train_labels)
test_labels_tensor = torch.tensor(test_labels)

In [ ]:
# Custom dataset class to handle tokenized data
class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # Tokenized text
        self.labels = labels  # Corresponding labels (0 = negative, 1 = positive)

    def __len__(self):
        return len(self.labels)  # Number of samples in dataset

    def __getitem__(self, idx):
        # Retrieve tokenized text and label for a given index
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

# Create dataset objects for training and testing
train_dataset = IMDBDataset(train_encodings, train_labels_tensor)
test_dataset = IMDBDataset(test_encodings, test_labels_tensor)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
# Load the pre-trained DistilBERT model for binary classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Move model to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Set model to evaluation mode (disables dropout and gradients)
model.eval()

predictions = []
true_labels = []

# Disable gradient computation to save memory during testing
with torch.no_grad():
    for batch in test_loader:
        batch = {key: val.to(device) for key, val in batch.items()}  # Move batch to device
        outputs = model(**batch)  # Forward pass
        logits = outputs.logits  # Get model outputs

        # Convert logits to class predictions (0 or 1)
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = batch["labels"].cpu().numpy()

        # Store predictions and actual labels for accuracy calculation
        predictions.extend(preds)
        true_labels.extend(labels)

# Compute accuracy score
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.4f}")

###**Using LIME to Explain Text:**

LIME can be used to explain the most important words when classifying text. Similar to the previous example, there are 3 main parts:

####**1) Define Classifier Function:**

  This function takes a list of d strings and outputs a (d, k) numpy array with prediction probabilities, where k is the number of classes.

####**2) Create Explainer**

  The LimeTextExplainer() will use one parameter:

  *  **class_names**
  
  list of class names, ordered according to whatever the classifier is using. If not present, class names will be ‘0’, ‘1’, …

####**3) Create Explanation**

  Call explain_instance() from the explainer. It will use the following parameters:

  * **text_instance**

  Raw text string to be explained.
  
  * **classifier_fn**

  Classifier function defined above.

  * **num_features**

  The number of words to be explained in the text.

In [ ]:
import numpy as np
from transformers import pipeline
from lime.lime_text import LimeTextExplainer

# Create a Hugging Face pipeline for classification
# This is a way to simplify the text classification process.
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True, device=0 if torch.cuda.is_available() else -1)

def predict_proba(texts):
    predictions = classifier(texts)
    return np.array([[p["score"] for p in pred] for pred in predictions])


In [ ]:
# Create LIME Explainer
explainer = LimeTextExplainer(class_names=["negative", "positive"])

# Pick a sample
idx = 0
sample_text = test_texts[idx]
true_label = test_labels[idx]

# Create explanation
explanation = explainer.explain_instance(
    sample_text,
    predict_proba,  # Function to get model probabilities
    num_features=10  # Number of words to explain
)

# Show results
print(f"True Label: {'positive' if true_label == 1 else 'negative'}")
explanation.show_in_notebook()


## **Part 4: Import Your Own Model**

Use the code below to import one of your own pre-trained model weights and use LIME. You will also need to import the dataset that this model was trained on.
If your dataset does not fit into one of the above examples, there is still likely a way to use LIME on your model. [See the documentation to find out how to implement it.](https://lime-ml.readthedocs.io/en/latest/lime.html)

In [ ]:
# Import model weights

from google.colab import files

# Upload a file
uploaded = files.upload()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize your model.
model = XXX.to(device)

# Load the weights into the model
model.load_state_dict(torch.load('model.pth'))

In [ ]:
# Load dataset and get predictions using pre-trained model

## Conclusion
You’ve now seen how to use LIME to explain predictions for tabular, text, and image-based models. LIME is a powerful tool to gain insight into the decision-making of your machine learning models!